In [ ]:
import os
import cv2
import numpy as np

In [ ]:
def fov_extractor(image):
    # Step 1: Convert to red channel and find image dimensions
    red_channel = image[:, :, 2]  # Assuming the image is in BGR format
    h, w = red_channel.shape

    # Step 2: Calculate center lines
    Hcenterline = h // 2
    Vcenterline = w // 2

    # Step 3: Draw scanning lines and calculate intensity profile
    horizontal_line = red_channel[Hcenterline, :]
    vertical_line = red_channel[:, Vcenterline]

    # Step 4: Calculate threshold using empirical factor
    th = max(horizontal_line) * 0.06

    # Find transitions based on threshold
    horizontal_transitions = np.where(np.diff(horizontal_line > th))[0]
    vertical_transitions = np.where(np.diff(vertical_line > th))[0]

    # Ensure there are at least two transitions to form a rectangle
    if len(horizontal_transitions) >= 2 and len(vertical_transitions) >= 2:
        X1, X2 = horizontal_transitions[[0, -1]]
        Y1, Y2 = vertical_transitions[[0, -1]]

        # Step 5: Crop the FOV based on found coordinates
        fov_image = image[Y1:Y2, X1:X2]
        return fov_image
    else:
        print("No valid transitions found")
        return None  # In case no valid transitions are found


def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)
            if image is not None:
                processed_image = fov_extractor(image)
                if processed_image is not None:
                    output_path = os.path.join(output_folder, filename)
                    cv2.imwrite(output_path, processed_image)
                else:
                    print(f"Processing failed for image: {filename}")
            else:
                print(f"Failed to load image: {filename}")
                


input_folder_path = "input_images"
output_folder_path = "output_images"
process_images(input_folder_path, output_folder_path)